In [68]:
from google_cloud_pipeline_components import aiplatform as gcc
from kfp.v2.dsl import pipeline

@pipeline(name='mpg-automl')
def pipeline(
    project: str
):
    vtx_dataset_job = gcc.TabularDatasetCreateOp(
        display_name = 'median_house_value',
        project = project,
        gcs_source = 'gs://vtx-datasets-public/mpg-custom/dataset.csv'
    )
    vtx_train_job = gcc.AutoMLTabularTrainingJobRunOp(
        display_name = 'automl-text-train-mpg',
        project = project,
        dataset = vtx_dataset_job.outputs["dataset"],
        optimization_prediction_type = 'regression',
        target_column = 'median_house_value'
    )
    create_endpoint_op = gcc.EndpointCreateOp(
        project = project,
        display_name = "automl-mpg-endpoint",
    )
    model_deploy_op = gcc.ModelDeployOp(
        model = vtx_train_job.outputs["model"],
        endpoint = create_endpoint_op.outputs['endpoint'],
        dedicated_resources_machine_type = 'n1-standard-4',
        dedicated_resources_min_replica_count = 1,
        dedicated_resources_max_replica_count = 1
    )

In [69]:
from kfp.v2 import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='vtx-automl-mpg.json')

In [70]:
import google.cloud.aiplatform as aip

job = aip.PipelineJob(
    display_name="kfp-v2",
    template_path="vtx-automl-mpg.json",
    pipeline_root="gs://vtx-pipe-root/",
    parameter_values={
        "project": "jchavezar-demo"
    },
)

job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/569083142710/locations/us-central1/pipelineJobs/mpg-automl-20221122131035
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/569083142710/locations/us-central1/pipelineJobs/mpg-automl-20221122131035')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mpg-automl-20221122131035?project=569083142710
